In [1]:
!pip install tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import cv2

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from sklearn import metrics

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)


Device: cuda


In [6]:
def is_valid_file(path):
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.webp')
    return path.lower().endswith(valid_extensions)


In [9]:
root_dir = '/kaggle/input/rice-image-dataset'
dataset = datasets.ImageFolder(root=root_dir, is_valid_file=is_valid_file)


In [11]:
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

Train: 60000, Val: 7500, Test: 7500


In [13]:
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(degrees=30),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset.dataset.transform = train_transforms
val_dataset.dataset.transform = val_test_transforms
test_dataset.dataset.transform = val_test_transforms

In [14]:
model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
for param in model.features.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(model.classifier[0].in_features, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Linear(32, 5)
)
model = model.to(DEVICE)


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 222MB/s]  


In [16]:
learning_rate = 3e-5
weight_decay = 1e-5
batch_size = 32

optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate, weight_decay=weight_decay)
loss_function = nn.CrossEntropyLoss()

In [18]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [20]:
epochs = 20
for epoch in range(epochs):
    model.train()
    for batch_features, batch_labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        batch_features = batch_features.to(DEVICE)
        batch_labels = batch_labels.to(DEVICE)

        y_preds = model(batch_features)
        loss = loss_function(y_preds.squeeze(), batch_labels)

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.classifier.parameters(), max_norm=1.0)
        optimizer.step()

    model.eval()
    y_probs, y_preds, y_test = [], [], []

    with torch.no_grad():
        for batch_features, batch_labels in tqdm(val_loader, desc="Validation"):
            batch_features = batch_features.to(DEVICE)
            batch_labels = batch_labels.to(DEVICE)

            model_probs = model(batch_features)
            y_probs += model_probs.tolist()
            y_test += batch_labels.tolist()

    y_probs = np.array(y_probs)
    y_test = np.array(y_test)
    y_preds = np.argmax(y_probs, axis=1)

    f1_score = metrics.f1_score(y_test, y_preds, average='macro')
    print(f"Epoch: {epoch+1} ~ F1 Score: {f1_score:.4f}")

Validation: 100%|██████████| 235/235 [01:35<00:00,  2.47it/s]


Epoch: 1 ~ F1 Score: 1.0000


Validation: 100%|██████████| 235/235 [01:03<00:00,  3.71it/s]


Epoch: 2 ~ F1 Score: 1.0000


Validation: 100%|██████████| 235/235 [01:03<00:00,  3.72it/s]


...

Epoch: 18 ~ F1 Score: 1.0000


Validation: 100%|██████████| 235/235 [01:05<00:00,  3.59it/s]


Epoch: 19 ~ F1 Score: 1.0000


Validation: 100%|██████████| 235/235 [01:05<00:00,  3.61it/s]

Epoch: 20 ~ F1 Score: 1.0000


In [1]:
from tqdm import tqdm
import numpy as np
import torch
model.eval()
all_probs = []
all_labels = []

with torch.no_grad():
    for batch_features, batch_labels in tqdm(test_loader, desc="Testing"):
        batch_features = batch_features.to(DEVICE)
        batch_labels = batch_labels.to(DEVICE)
        outputs = model(batch_features)
        all_probs.append(outputs.cpu())
        all_labels.append(batch_labels.cpu())
y_probs = torch.cat(all_probs).numpy()
y_test = torch.cat(all_labels).numpy()
y_preds = np.argmax(y_probs, axis=1)

NameError: name 'model' is not defined

In [2]:
print("Accuracy:", metrics.accuracy_score(y_test, y_preds))
print("Precision:", metrics.precision_score(y_test, y_preds, average='macro'))
print("Recall:", metrics.recall_score(y_test, y_preds, average='macro'))
print("F1 Score:", metrics.f1_score(y_test, y_preds, average='macro'))

NameError: name 'metrics' is not defined